Colab is making it easier than ever to integrate powerful Generative AI capabilities into your projects. We are launching public preview for a simple and intuitive Python library (google.colab.ai) to access state-of-the-art language models directly within Colab environments. All users have free access to most popular LLMs, while paid users have access to a wider selection of models. This means users can spend less time on configuration and set up and more time bringing their ideas to life. With just a few lines of code, you can now perform a variety of tasks:
- Generate text
- Translate languages
- Write creative content
- Categorise text

In [1]:
# @title List available models
from google.colab import ai

ai.list_models()

['google/gemini-2.5-flash', 'google/gemini-2.5-flash-lite']

Choosing a Model
The model names give you a hint about their capabilities and intended use:

Pro: These are the most capable models, ideal for complex reasoning, creative tasks, and detailed analysis.

Flash: These models are optimized for high speed and efficiency, making them great for summarization, chat applications, and tasks requiring rapid responses.

Gemma: These are lightweight, open-weight models suitable for a variety of text generation tasks and are great for experimentation.

In [2]:
# @title Simple batch generation example
# Only text-to-text input/output is supported
from google.colab import ai

response = ai.generate_text("What is the capital of France?")
print(response)

The capital of France is **Paris**.


In [3]:
# @title Choose a different model
from google.colab import ai

response = ai.generate_text("What is the capital of England", model_name='google/gemini-2.5-flash-lite')
print(response)

The capital of England is **London**.


For longer text generations, you can stream the response. This displays the output token by token as it's generated, rather than waiting for the entire response to complete. This provides a more interactive and responsive experience. To enable this, simply set stream=True.

In [4]:
# @title Simple streaming example
from google.colab import ai

stream = ai.generate_text("Tell me a short story.", stream=True)
for text in stream:
  print(text, end='')

Elara lived a life painted in muted tones. Her apartment, though cozy, was filled with the quiet hum of routine: morning tea, afternoon reading by the window, evenings spent tending her small balcony garden. She found solace in the predictable, but sometimes, a whisper of longing for something more vibrant would stir within her.

One blustery Tuesday, seeking refuge from an unexpected downpour, Elara ducked into a cluttered antique shop she'd passed a hundred times but never entered. Dust motes danced in the dim light, illuminating forgotten treasures. Her gaze fell upon a small, wooden music box tucked away on a lower shelf. It was unassuming, plain, with no intricate carvings or bright paint. Yet, something about its simplicity called to her.

She bought it for a pittance, the shopkeeper barely looking up from his crossword. Back home, she placed it on her reading table, admiring its smooth, worn surface. It had no key, no visible crank. It seemed utterly silent. A momentary disappoi

In [6]:
#@title Text formatting setup
#code is not necessary for colab.ai, but is useful in fomatting text chunks
import sys

class LineWrapper:
    def __init__(self, max_length=80):
        self.max_length = max_length
        self.current_line_length = 0

    def print(self, text_chunk):
        i = 0
        n = len(text_chunk)
        while i < n:
            start_index = i
            while i < n and text_chunk[i] not in ' \n': # Find end of word
                i += 1
            current_word = text_chunk[start_index:i]

            delimiter = ""
            if i < n: # If not end of chunk, we found a delimiter
                delimiter = text_chunk[i]
                i += 1 # Consume delimiter

            if current_word:
                needs_leading_space = (self.current_line_length > 0)

                # Case 1: Word itself is too long for a line (must be broken)
                if len(current_word) > self.max_length:
                    if needs_leading_space: # Newline if current line has content
                        sys.stdout.write('\n')
                        self.current_line_length = 0
                    for char_val in current_word: # Break the long word
                        if self.current_line_length >= self.max_length:
                            sys.stdout.write('\n')
                            self.current_line_length = 0
                        sys.stdout.write(char_val)
                        self.current_line_length += 1
                # Case 2: Word doesn't fit on current line (print on new line)
                elif self.current_line_length + (1 if needs_leading_space else 0) + len(current_word) > self.max_length:
                    sys.stdout.write('\n')
                    sys.stdout.write(current_word)
                    self.current_line_length = len(current_word)
                # Case 3: Word fits on current line
                else:
                    if needs_leading_space:
                        # Define punctuation that should not have a leading space
                        # when they form an entire "word" (token) following another word.
                        no_leading_space_punctuation = {
                            ",", ".", ";", ":", "!", "?",        # Standard sentence punctuation
                            ")", "]", "}",                     # Closing brackets
                            "'s", "'S", "'re", "'RE", "'ve", "'VE", # Common contractions
                            "'m", "'M", "'ll", "'LL", "'d", "'D",
                            "n't", "N'T",
                            "...", "…"                          # Ellipses
                        }
                        if current_word not in no_leading_space_punctuation:
                            sys.stdout.write(' ')
                            self.current_line_length += 1
                    sys.stdout.write(current_word)
                    self.current_line_length += len(current_word)

            if delimiter == '\n':
                sys.stdout.write('\n')
                self.current_line_length = 0
            elif delimiter == ' ':
                # If line is full and a space delimiter arrives, it implies a wrap.
                if self.current_line_length >= self.max_length:
                    sys.stdout.write('\n')
                    self.current_line_length = 0

        sys.stdout.flush()


In [7]:
# @title Formatted streaming example
from google.colab import ai

wrapper = LineWrapper()
for chunk in ai.generate_text('Give me a long winded description about the evolution of the Roman Empire.', model_name='google/gemini-2.5-flash', stream=True):
  wrapper.print(chunk)

From the humble seed planted on the Palatine Hill, a tiny settlement of
audacious, pragmatic, and remarkably tenacious Latin tribesmen, the Roman Empire
burgeoned into the most enduring and influential political entity of the Western
world. Its evolution was a sprawling, millennia-long saga, a testament to
adaptability, ambition, and, ultimately, the inescapable pull of entropy. It
wasn't a static monument, but a living, breathing, constantly shifting organism,
forever remaking itself, often violently, until its very essence was
transformed.

**The Nascent Republic: From Monarchy to the Struggle of the Orders (c. 753 BCE
- 27 BCE)**

Rome's earliest legends speak of a monarchy, founded by Romulus and nurtured by
a succession of kings, some native Latin, others Etruscan. This period, shrouded
in myth and faint historical whispers, laid foundational elements: a strong
military tradition, a hierarchical social structure, and a deep-seated distrust
of absolute power. The expulsion of the l